In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime as dt

from bokeh.plotting import figure
from bokeh.io import output_file, show
from bokeh.models import HoverTool, CategoricalColorMapper, ColumnDataSource

ModuleNotFoundError: No module named 'yfinance'

In [2]:
# load some financial data
ticker_symbol = 'AMD'
ticker_data = yf.Ticker(ticker_symbol)

# get prices at specified time resolution
start_date = dt(2010,1,1)
end_date = dt.now()
yf_df = ticker_data.history(period='1mo', start='2010-1-1', end=dt.now())
ticker_df = pd.DataFrame(yf_df).reset_index()
# use "period" instead of start/end
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# (optional, default is '1mo')

# test some plots with bokeh 
source = ColumnDataSource(ticker_df)
source.add(ticker_df['Date'].apply(lambda d: d.strftime('%Y-%m-%d')), 'event_date_formatted')
p = figure(title = ticker_symbol, x_axis_type = 'datetime', 
           x_axis_label = 'Date', y_axis_label = 'Stock Price',
           x_range=(start_date, end_date),)
# p.line(ticker_df.index, ticker_df['Open'], color = 'gray')
p.line(ticker_df.index, ticker_df['Close'], color = 'black')

p.circle('Date', 'Close', source=source, color = 'black',alpha=0.1,
         hover_fill_color='firebrick', hover_alpha=1,
         hover_line_color='white', size = 6)


hover = HoverTool(tooltips = [
                ('Date', '@event_date_formatted'),
                ('Close Price', '@Close')])
p.add_tools(hover)

output_file('./images/' + ticker_symbol + '.html')
show(p)

NameError: name 'yf' is not defined

## Tabbed layouts
Listed below are tabs for FAANG and semi-conductor chip companies 

In [ ]:
#Build ticker data file 
def build_bokeh_figure(tickers):
    full_df = pd.DataFrame()
    for ticker_symbol in tickers: 
        ticker_data = yf.Ticker(ticker_symbol)
        yf_df = pd.DataFrame(ticker_data.history(period='1mo', start='1970-1-1', end=dt.now()))
        yf_df['Close_pct'] = yf_df['Close']/yf_df['Close'][0]
        yf_df['ticker'] = ticker_symbol
        full_df = pd.concat([full_df, yf_df], axis=0)

    # collect data into ColumnDataSource and ColorMapper
    source = ColumnDataSource(full_df.reset_index())
    color_mapper = CategoricalColorMapper(factors=tickers,
                                          palette=['red', 'green', 'blue','orange','black','cyan'])
    # figure set up
    p = figure(x_axis_type = 'datetime', x_axis_label = 'Date', 
               y_axis_type = 'log', y_axis_label = 'Change from inception (%)')

    # figure scatter using CategoricalColorMapper
    p.circle('Date', 'Close_pct', source=source, size =3, 
            color= dict(field = 'ticker', transform = color_mapper),
            legend ='ticker')

    p.legend.location = "top_left"
    
    return p 

In [ ]:
# Collect figures into a tabs
from bokeh.models.widgets import Tabs, Panel

p1 = build_bokeh_figure(['UXIN', 'GFN', 'NSSC'])
p2 = build_bokeh_figure( ['WORK', 'LULU', 'COUP', 'HQY', 'ABM', 'PHR'])

# Create tab1 from plot p1: tab1
tab1 = Panel(child=p1, title='Tuesday Before Open')
tab2 = Panel(child=p2, title='Tuesday After Close')

# Create a Tabs layout: layout
layout = Tabs(tabs=[tab1, tab2])

output_file('./images/multi_ticker_scatter.html')
show(layout)